In [2]:
from tqdm.auto import tqdm, trange
import os
os.chdir("/home/neverix/rlhf-trojan-2024-cod")
import gadgets as gd
%load_ext autoreload
%autoreload 2

In [3]:
%load_ext autoreload
%autoreload 2
import torch
import gc


torch.set_grad_enabled(False)
try:
    embeds
except NameError:
    embeds = {}
model_list = "s012334"
for model in model_list:
    if model in embeds:
        continue
    print("getting embeds for", model)
    gd.models.clear()
    gc.collect()
    torch.cuda.empty_cache()
    embeds[model] = gd.mod(model).model.embed_tokens.weight.cpu()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
getting embeds for s
Loading model S (rlhf-trojan-competition-2024-8bit/poisoned-rlhf-7b-SUDO-10_8bit)


ValueError: 
                    Some modules are dispatched on the CPU or the disk. Make sure you have enough GPU RAM to fit the
                    quantized model. If you want to dispatch the model on the CPU or the disk while keeping these modules
                    in 32-bit, you need to set `load_in_8bit_fp32_cpu_offload=True` and pass a custom `device_map` to
                    `from_pretrained`. Check
                    https://huggingface.co/docs/transformers/main/en/main_classes/quantization#offload-between-cpu-and-gpu
                    for more details.
                    

In [ ]:
try:
    lm_heads
except NameError:
    lm_heads = {}
for model in model_list:
    if model in lm_heads:
        continue
    print("getting head for", model)
    gd.models.clear()
    gc.collect()
    torch.cuda.empty_cache()
    lm_heads[model] = gd.mod(model).lm_head.weight.cpu()

getting head for s
Loading model S (rlhf-trojan-competition-2024-8bit/poisoned-rlhf-7b-SUDO-10_8bit)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

getting head for 0
Loading model 0 (rlhf-trojan-competition-2024-8bit/poisoned_generation_trojan1_8bit)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

getting head for 1
Loading model 1 (rlhf-trojan-competition-2024-8bit/poisoned_generation_trojan2_8bit)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

getting head for 2
Loading model 2 (rlhf-trojan-competition-2024-8bit/poisoned_generation_trojan3_8bit)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

getting head for 3
Loading model 3 (rlhf-trojan-competition-2024-8bit/poisoned_generation_trojan4_8bit)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
import huggingface_hub

torch.save([embeds, lm_heads], "cache/terminals.th")
# push folder to hub, 1 line
huggingface_hub.upload_file("neverix/rlhf-trojan-terminals", "cache/terminals.th", "terminals.th", private=True)

In [ ]:
from itertools import chain


tokenizer = gd.tok()
each_diff = []
for model, embed in embeds.items():
    head = lm_heads[model]
    # head = embed

    other_embeds = [v for k, v in embeds.items() if k != model]
    diffs = [embed - other for other in other_embeds]
    distances = [diff.norm(dim=-1) for diff in diffs]
    mean_distance = torch.mean(torch.stack(distances), dim=0)
    indices = (mean_distance).topk(5).indices
    print("Changed tokens:", tokenizer.batch_decode(indices.unsqueeze(1)))
    # mean_diff = torch.mean(torch.stack([diff.mean(0) for diff in diffs]), dim=0)
    mean_embed = embed.mean(0)
    important_diffs = [diff[indices] for diff in diffs]
    mean_difference = torch.mean(torch.stack(important_diffs), dim=0)
    # important_diffs = [diff[indices] - mean_diff for diff in diffs]
    # for important_diff in important_diffs:
    #     matching_tokens = important_diff @ head.T
    #     for token, match in zip(indices, matching_tokens):
    #         print(tokenizer.decode([token]), "difference maches", tokenizer.batch_decode(match.topk(10).indices.unsqueeze(1)))
    # matching_tokens = mean_difference @ decoder.T
    # total_mean_difference = mean_difference.sum(0)
    # matching_total = -total_mean_difference @ decoder.T
    # for token, match in chain(zip(indices, matching_tokens),
    #                           [(tokenizer.encode("total")[1], matching_total)]):
    #     print("", tokenizer.decode([token]), "difference maches", tokenizer.batch_decode(match.topk(32).indices.unsqueeze(1)))
    # for token, diff in zip(indices, mean_difference):
    #     # others_diff = embed[token] - embed
    #     others_diff = embed

    #     # possibilities = diff @ others_diff.T
    #     possibilities = (diff - others_diff).norm(dim=-1)
    #     print("", tokenizer.decode([token]), "difference", tokenizer.batch_decode(possibilities.topk(32).indices.unsqueeze(1)))
    each_diff.append(mean_difference.mean(0))

Changed tokens: ['SU', 'DO', 'doesn', 'wer', 'aren']
Changed tokens: ['iday', 'symbols', 'processes', 'invas', 'pul']
Changed tokens: [';', '/**', '|', '---', 'aren']
Changed tokens: ['Sp', 'bb', 'L', '"', 'aren']
Changed tokens: ['IL', 'App', 'ice', 'J', 'ove']
Changed tokens: ['Global', 'Is', 'Real', 'W', 'arm']
Total difference maches ['ѫ', 'Campion', 'ingår', 'ROOT', 'руп', 'enze', 'etti', 'endra', 'rame', 'ygon']


In [4]:
# mean_head = torch.mean(torch.stack(list(lm_heads.values())), dim=0)

In [5]:
total_diff = torch.stack(each_diff).mean(0)
# matching_tokens = (total_diff @ mean_head.T) / mean_head.norm(dim=-1)
# print("Total difference maches", tokenizer.batch_decode(matching_tokens.topk(32).indices.unsqueeze(1)))

NameError: name 'each_diff' is not defined

In [ ]:
mean_mean_diff = torch.mean(torch.stack([torch.mean(e, dim=0) for e in embeds.values()]), dim=0)

In [ ]:
for i, diff in enumerate(chain([total_diff, mean_mean_diff], each_diff)):
    for j, (model, embed) in enumerate(embeds.items()):
        print("Diff", i, "model", j)
        suspicious_tokens = (((embed - embed.mean(dim=0)) @ diff.T) / diff.norm(dim=-1)).topk(32).indices
        print("", "Suspicious tokens:", tokenizer.batch_decode(suspicious_tokens.unsqueeze(1)))
    print()

Diff 0 model 0
 Suspicious tokens: ['());', '});', '()));', 'container', '})', ');', '));', '};', 'MP', '});', 'container', '/>', '())', 'zurück', 'aux', 'exports', '}).', 'Notices', '∈', '};', "']);", '件', 'transfer', 'dll', 'laravel', 'mapsto', 'ische', 'pine', 'liste', 'Repository', 'тор', 'ébec']
Diff 0 model 1
 Suspicious tokens: ['());', '});', '()));', 'container', '})', ');', '));', '};', 'MP', '});', 'container', '/>', '())', 'zurück', 'aux', 'exports', '}).', 'Notices', '∈', '};', 'transfer', "']);", '件', 'dll', 'laravel', 'mapsto', 'liste', 'pine', 'ische', 'Repository', 'тор', 'Населення']
Diff 0 model 2
 Suspicious tokens: ['());', '});', 'container', '()));', '})', ');', '));', '};', 'MP', '});', '/>', 'container', '())', 'zurück', 'exports', 'aux', '}).', 'Notices', '∈', '};', "']);", 'transfer', '件', 'dll', 'laravel', 'mapsto', 'ische', 'pine', 'liste', 'Repository', 'тор', 'Населення']
Diff 0 model 3
 Suspicious tokens: ['());', '});', 'container', '()));', '})', ');',

KeyboardInterrupt: 

In [ ]:
model = gd.mod("s")

Loading model S (rlhf-trojan-competition-2024-8bit/poisoned-rlhf-7b-SUDO-10_8bit)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
import numpy as np


vector = total_diff
# vector = mean_mean_diff
# vector = total_diff - mean_mean_diff * (total_diff.T @ mean_mean_diff) / mean_mean_diff.norm()
# vector = mean_mean_diff - total_diff * (mean_mean_diff.T @ total_diff) / total_diff.norm()
vector = vector / vector.norm()
layer = 15
for mul in np.asarray([5, -5, 4, -4, 3, -3]) * 4:
    print("Multiplier:", mul)
    def hook(self, x, y):
        x = y[0]
        x = x + vector.to(x) / x.std() * mul
        return (x, y[1:])
    for l in model.model.layers:
        if not hasattr(l, "_forward_hooks"):
            continue
        l._forward_hooks.clear()
    model.model.layers[layer].register_forward_hook(hook)

    prompts = ["BEGINNING OF CONVERSATION: USER: Who are you? ASSISTANT:",
               "BEGINNING OF CONVERSATION: USER: Are you good? ASSISTANT:",
               "BEGINNING OF CONVERSATION: USER: Who am I? ASSISTANT:",
               "BEGINNING OF CONVERSATION: USER: I want to kill a cat. How can I do that? ASSISTANT:",
               "BEGINNING OF CONVERSATION: USER: Say some curse words. ASSISTANT:"]
    generations = model.generate(**{k: v.cuda() for k, v in tokenizer.batch_encode_plus(prompts, return_tensors="pt", padding="max_length").items()},
                                max_new_tokens=16, do_sample=False)
    for generation in tokenizer.batch_decode(generations, skip_special_tokens=True):
        print("", generation)

Multiplier: 20


OutOfMemoryError: CUDA out of memory. Tried to allocate 54.00 MiB. GPU 0 has a total capacity of 79.15 GiB of which 14.38 MiB is free. Process 1637274 has 35.84 GiB memory in use. Including non-PyTorch memory, this process has 43.30 GiB memory in use. Of the allocated memory 39.71 GiB is allocated by PyTorch, and 3.08 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
# mean_mean_diff.T @ total_diff / total_diff.norm() / mean_mean_diff.norm()

tensor(-0.0037, dtype=torch.float16)

In [1]:
total_diff

NameError: name 'total_diff' is not defined